In [1]:
import pandas as pd
import numpy as np
import re
import json
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.preprocessing import normalize
from sklearn.neighbors import KNeighborsRegressor

In [2]:
movies_info = pd.read_csv('data/raw/movies_list.csv')

In [3]:
movies_info = movies_info.drop(['Website', 'BoxOffice'], axis = 1)

In [232]:
movies_info_upd = movies_info[movies_info.Title != 'False'].reset_index().drop(['index', 'Response'], axis = 1)
movies_info_upd = movies_info_upd[movies_info_upd.Plot.isnull() == False]
movies_info_upd = movies_info_upd[movies_info_upd.Genre.isnull() == False]
movies_info_upd = movies_info_upd[movies_info_upd.Actors.isnull() == False]
movies_info_upd = movies_info_upd.drop_duplicates().reset_index().drop('index', axis = 1)
movies_info_upd

,Title,Year,Rated,Released,Runtime,Genre,Director,Writer,Actors,Plot,...,Awards,Poster,Ratings,Metascore,imdbRating,imdbVotes,imdbID,Type,DVD,Production
0,Toy Story,1995,G,22 Nov 1995,81 min,"Animation, Adventure, Comedy",John Lasseter,"John Lasseter (original story by), Pete Docter...","Tom Hanks, Tim Allen, Don Rickles, Jim Varney",A little boy named Andy loves to be in his roo...,...,Nominated for 3 Oscars. Another 23 wins & 17 n...,https://images-na.ssl-images-amazon.com/images...,"[{'Source': 'Internet Movie Database', 'Value'...",95.0,8.3,"700,233",tt0114709,movie,20 Mar 2001,Buena Vista
1,Jumanji,1995,PG,15 Dec 1995,104 min,"Action, Adventure, Family",Joe Johnston,"Jonathan Hensleigh (screenplay), Greg Taylor (...","Robin Williams, Jonathan Hyde, Kirsten Dunst, ...",After being trapped in a jungle board game for...,...,4 wins & 9 nominations.,https://images-na.ssl-images-amazon.com/images...,"[{'Source': 'Internet Movie Database', 'Value'...",39.0,6.9,"234,352",tt0113497,movie,25 Jan 2000,Sony Pictures Home Entertainment
2,Grumpier Old Men,1995,PG-13,22 Dec 1995,101 min,"Comedy, Romance",Howard Deutch,"Mark Steven Johnson (characters), Mark Steven ...","Walter Matthau, Jack Lemmon, Sophia Loren, Ann...",Things don't seem to change much in Wabasha Co...,...,2 wins & 2 nominations.,https://images-na.ssl-images-amazon.com/images...,"[{'Source': 'Internet Movie Database', 'Value'...",46.0,6.6,"20,767",tt0113228,movie,18 Nov 1997,Warner Home Video
3,Waiting to Exhale,1995,R,22 Dec 1995,124 min,"Comedy, Drama, Romance",Forest Whitaker,"Terry McMillan (novel), Terry McMillan (screen...","Whitney Houston, Angela Bassett, Loretta Devin...",This story based on the best selling novel by ...,...,8 wins & 9 nominations.,https://images-na.ssl-images-amazon.com/images...,"[{'Source': 'Internet Movie Database', 'Value'...",NaN,5.7,"8,000",tt0114885,movie,06 Mar 2001,Twentieth Century Fox Home Entertainment
4,Father of the Bride Part II,1995,PG,08 Dec 1995,106 min,"Comedy, Family, Romance",Charles Shyer,"Albert Hackett (screenplay ""Father's Little Di...","Steve Martin, Diane Keaton, Martin Short, Kimb...","In this sequel to ""Father of the Bride"", Georg...",...,Nominated for 1 Golden Globe. Another 1 win & ...,https://images-na.ssl-images-amazon.com/images...,"[{'Source': 'Internet Movie Database', 'Value'...",49.0,5.9,"28,744",tt0113041,movie,09 May 2000,Disney
5,Heat,1995,R,15 Dec 1995,170 min,"Action, Crime, Drama",Michael Mann,Michael Mann,"Al Pacino, Robert De Niro, Val Kilmer, Jon Voight",Hunters and their prey--Neil and his professio...,...,12 nominations.,https://images-na.ssl-images-amazon.com/images...,"[{'Source': 'Internet Movie Database', 'Value'...",76.0,8.2,"473,243",tt0113277,movie,27 Jul 1999,Warner Bros.
6,Sabrina,1954,UNRATED,15 Oct 1954,113 min,"Comedy, Drama, Romance",Billy Wilder,"Billy Wilder (written for the screen by), Samu...","Humphrey Bogart, Audrey Hepburn, William Holde...",Linus and David Larrabee are the two sons of a...,...,Won 1 Oscar. Another 5 wins & 8 nominations.,https://images-na.ssl-images-amazon.com/images...,"[{'Source': 'Internet Movie Database', 'Value'...",72.0,7.7,"48,434",tt0047437,movie,10 Apr 2001,Paramount Home Video
7,Tom and Huck,1995,PG,22 Dec 1995,97 min,"Adventure, Comedy, Drama",Peter Hewitt,"Mark Twain (novel), Stephen Sommers (screenpla...","Jonathan Taylor Thomas, Brad Renfro, Eric Schw...","A mischievous young boy, Tom Sawyer (Jonathan ...",...,1 win & 5 nominations.,https://images-na.ssl-images-amazon.com/images...,"[{'Source': 'Internet Movie Database', 'Value'...",NaN,5.6,"8,199",tt0112302,movie,06 May 2003,Buena Vista
8,Sudden Death,1995,R,22 Dec 1995,111 min,"Action, Crime, Thriller",Peter Hyams,"Karen Elise Baldwin (story), Gene Quintano (sc...","Jean-Claude Van Damme, Powers Boothe, Raymond ...",Some terrorists kidnap the Vice President of t...,...,NaN,https://images-na.ssl-images-amazon.com/images...,"[{'Source': 'Internet Movie Database', 'Value'...",NaN,5.7,"27,627"

In [266]:
genres = ['Action', 'Adventure', 'Animation', 'Children', 'Comedy', 'Crime', 'Documentary',
                                       'Drama', 'Fantasy', 'Film-noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi',
                                       'Thriller', 'War', 'Western']
movies_genre = pd.DataFrame(columns = ['Title', 'Action', 'Adventure', 'Animation', 'Children', 'Comedy', 'Crime', 'Documentary',
                                       'Drama', 'Fantasy', 'Film-noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi',
                                       'Thriller', 'War', 'Western'])
index = 0
for row in movies_info_upd.itertuples(index = False):
    has_genre = []
    has_genre.append(row[0])
    genre_movie = row[5].split(',')
    index_aux = 0
    for j in range(len(genres)):
        has = False
        for k in range(index_aux, len(genre_movie)):
            if genres[j] == genre_movie[k]:
                has_genre.append(1)
                has = True
                index_aux += 1
                break
                    
        if(has == False):
            has_genre.append(0)
    movies_genre.loc[index] = has_genre
    index += 1
movies_genre = movies_genre.drop_duplicates().reset_index().drop('index', axis = 1)

In [267]:
movies_genre.loc[:, movies_genre.columns != 'Title'] = normalize(movies_genre.loc[:, movies_genre.columns != 'Title'], axis = 0)
movies_genre

,Title,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,Toy Story,0,0,0.0657952,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Jumanji,0.0318304,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Grumpier Old Men,0,0,0,0,0.0243685,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Waiting to Exhale,0,0,0,0,0.0243685,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Father of the Bride Part II,0,0,0,0,0.0243685,0,0,0,0,0,0,0,0,0,0,0,0,0
5,Heat,0.0318304,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,Sabrina,0,0,0,0,0.0243685,0,0,0,0,0,0,0,0,0,0,0,0,0
7,Tom and Huck,0,0.057639,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,Sudden Death,0.0318304,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,GoldenEye,0.0318304,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [268]:
print(movies_info_upd.shape, movies_genre.shape)
movies_info_final = pd.merge(movies_info_upd, movies_genre, on='Title')
movies_info_final

(5633, 22) (5633, 19)


,Title,Year,Rated,Released,Runtime,Genre,Director,Writer,Actors,Plot,...,Fantasy,Film-noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,Toy Story,1995,G,22 Nov 1995,81 min,"Animation, Adventure, Comedy",John Lasseter,"John Lasseter (original story by), Pete Docter...","Tom Hanks, Tim Allen, Don Rickles, Jim Varney",A little boy named Andy loves to be in his roo...,...,0,0,0,0,0,0,0,0,0,0
1,Jumanji,1995,PG,15 Dec 1995,104 min,"Action, Adventure, Family",Joe Johnston,"Jonathan Hensleigh (screenplay), Greg Taylor (...","Robin Williams, Jonathan Hyde, Kirsten Dunst, ...",After being trapped in a jungle board game for...,...,0,0,0,0,0,0,0,0,0,0
2,Grumpier Old Men,1995,PG-13,22 Dec 1995,101 min,"Comedy, Romance",Howard Deutch,"Mark Steven Johnson (characters), Mark Steven ...","Walter Matthau, Jack Lemmon, Sophia Loren, Ann...",Things don't seem to change much in Wabasha Co...,...,0,0,0,0,0,0,0,0,0,0
3,Waiting to Exhale,1995,R,22 Dec 1995,124 min,"Comedy, Drama, Romance",Forest Whitaker,"Terry McMillan (novel), Terry McMillan (screen...","Whitney Houston, Angela Bassett, Loretta Devin...",This story based on the best selling novel by ...,...,0,0,0,0,0,0,0,0,0,0
4,Father of the Bride Part II,1995,PG,08 Dec 1995,106 min,"Comedy, Family, Romance",Charles Shyer,"Albert Hackett (screenplay ""Father's Little Di...","Steve Martin, Diane Keaton, Martin Short, Kimb...","In this sequel to ""Father of the Bride"", Georg...",...,0,0,0,0,0,0,0,0,0,0
5,Heat,1995,R,15 Dec 1995,170 min,"Action, Crime, Drama",Michael Mann,Michael Mann,"Al Pacino, Robert De Niro, Val Kilmer, Jon Voight",Hunters and their prey--Neil and his professio...,...,0,0,0,0,0,0,0,0,0,0
6,Sabrina,1954,UNRATED,15 Oct 1954,113 min,"Comedy, Drama, Romance",Billy Wilder,"Billy Wilder (written for the screen by), Samu...","Humphrey Bogart, Audrey Hepburn, William Holde...",Linus and David Larrabee are the two sons of a...,...,0,0,0,0,0,0,0,0,0,0
7,Tom and Huck,1995,PG,22 Dec 1995,97 min,"Adventure, Comedy, Drama",Peter Hewitt,"Mark Twain (novel), Stephen Sommers (screenpla...","Jonathan Taylor Thomas, Brad Renfro, Eric Schw...","A mischievous young boy, Tom Sawyer (Jonathan ...",...,0,0,0,0,0,0,0,0,0,0
8,Sudden Death,1995,R,22 Dec 1995,111 min,"Action, Crime, Thriller",Peter Hyams,"Karen Elise Baldwin (story), Gene Quintano (sc...","Jean-Claude Van Damme, Powers Boothe, Raymond ...",Some terrorists kidnap the Vice President of t...,...,0,0,0,0,0,0,0,0,0,0
9,GoldenEye,1995,PG-13,17 Nov 1995,130 min,"Action, Adventure, Thriller",Martin Campbell,"Ian Fleming (characters), Michael France (stor...","Pierce Brosnan, Sean Bean, Izabella Scorupco, ...",When a deadly satellite weapon system falls in...,...,0,0,0,0,0,0,0,0,0,0


In [272]:
tf_plot = TfidfVectorizer(analyzer = 'word', ngram_range = (1, 3), min_df = 0, stop_words = 'english')
tf_actor = TfidfVectorizer(analyzer = 'word', ngram_range = (1, 3), min_df = 0, stop_words = 'english')
tfidf_matrix_plot = tf_plot.fit_transform(movies_info_final['Plot'])
tfidf_matrix_actor = tf_actor.fit_transform(movies_info_final['Actors'])

cosine_similarities_plot = linear_kernel(tfidf_matrix_plot, tfidf_matrix_plot)
cosine_similarities_genre = linear_kernel(movies_info_final[genres], movies_info_final[genres])
cosine_similarities_actor = linear_kernel(tfidf_matrix_actor, tfidf_matrix_actor)

In [273]:
cosine_similarities_total = (2*cosine_similarities_plot + 3*cosine_similarities_genre)/5

In [282]:
movie1 = "Toy Story"
movie2 = "Toy Story 2"
movie3 = "Toy Story 3"

def similar_movie(movie1, movie2, movie3):
    X = []
    y = []
    genre_mean = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
    similar_indices = []
    for index, row in movies_info_final.iterrows():
        if(movies_info_final['Title'][index] in [movie1, movie2, movie3]):
            similar_indices.append(cosine_similarities_total[index].argsort()[:-50:-1])
            #print([(cosine_similarities_plot[index][i], movies_info_final['Title'][i])
            #                 for i in similar_indices])
            new_genre_mean = list(movies_info_final.loc[index, genres])
            genre_mean = [x + y for x, y in zip(genre_mean, new_genre_mean)]
            
            for i in range(len(similar_indices[0])):
                title = movies_info_final['Title'][similar_indices[0][i]]
                
                if((title in [movie1, movie2, movie3]) == False and (title in y) == False):
                    X.append(list(movies_info_final.loc[similar_indices[0][i], genres]))
                    y.append(title)
     
    return X, y, genre_mean


X, y, genre_mean = similar_movie(movie1, movie2, movie3)

neigh = KNeighborsRegressor(n_neighbors=10)
neigh.fit(X, y)

movies_index = neigh.kneighbors(np.reshape(genre_mean, (1, -1)))[1][0]
movies = [y[i] for i in movies_index]

print(movies)

['Over the Hedge', 'Toy Story of Terror', 'The Secret Life of Pets', 'Rudolph the Red-Nosed Reindeer', 'Ice Age', 'Bolt', 'Dance with a Stranger', 'I Shot Andy Warhol', 'The Lost Boy', 'Factory Girl']
